In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import sys
import natsort
import GrowthAssayPlotlib

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
USE_MULTIINVASION = False

if USE_MULTIINVASION:
    ga2_file = 'GA2 parasitemia multiinvasion.xlsx'
    column_title_row = 0
    OUTPUT_FOLDER = 'Graph Multiinvasion'

else:
    ga2_file = 'GA2 parasitemia.xlsx'
    column_title_row = 3
    OUTPUT_FOLDER = ''

df2 = GrowthAssayPlotlib.load_file(ga2_file, column_title_row)
df2_melted_replicas = GrowthAssayPlotlib.melt_df(df2, id_vars=['Plate number', 'Well position', 'Line', 'Clone', 'Strain', 'Speed'])

hematocrit_levels = ['2%', '4%', '8%']

# Extract hematocrit level and create 'Line simple' column

df2_melted_replicas['Hematocrit'] = df2_melted_replicas['Line'].str.extract(f'({"|".join(hematocrit_levels)})', expand=False)
df2_melted_replicas['Hematocrit'] = df2_melted_replicas['Hematocrit'].fillna('4%')

group_keys = ['Line', 'Speed', 'Hematocrit']

df2_melted_replicas.head()
df2_melted_replicas['Line'].unique()
df2_melted_replicas['Hematocrit'].unique()

In [ ]:
# plot all 
MAX_INVASION_RATE_PLOT = 25
output_folder_ga2 = GrowthAssayPlotlib.get_output_folder('GA2', OUTPUT_FOLDER)

df2_melted_replicas_filtered = GrowthAssayPlotlib.filter_dataframe(
    df=df2_melted_replicas,
    ignore_repeats=[
        'Repeat 1',
    ],
    filter_list_filename='GA2 filtered.csv'
)

# group replicas per repeat
df2_melted_grouped = GrowthAssayPlotlib.group_repeats(
    df_melted_replicas_filtered = df2_melted_replicas_filtered,
    group_keys=['Line', 'Speed', 'Repeat', 'Strain', 'Clone', 'Hematocrit'],
)

df2_melted = df2_melted_replicas_filtered

df2_melted.to_json('GA2_df_melted.json')

# GrowthAssayPlotlib.plot_repeats_for_line(df2_melted_replicas_filtered, MAX_INVASION_RATE_PLOT, output_folder_ga2)
# GrowthAssayPlotlib.plot_repeats_for_wells(df2_melted_replicas_filtered, MAX_INVASION_RATE_PLOT, output_folder_ga2)
# GrowthAssayPlotlib.plot_parasimetia_vs_invasion_rate(df2_melted_replicas, ['Line', 'Speed'], output_folder=output_folder_ga2)
# for label, df_plot in [('with replicas', df2_melted_replicas), ('averaged replicas', df2_melted)]:
#     GrowthAssayPlotlib.plot_line(df_plot, label, MAX_INVASION_RATE_PLOT, output_folder_ga2)
#     GrowthAssayPlotlib.plot_speed(df_plot, label, MAX_INVASION_RATE_PLOT, output_folder_ga2)

In [ ]:
# GrowthAssayPlotlib.plot_pretty_barplot(
#     df=df2_melted,
#     plot_clones=False, # or 'Line' if you want clones to show separatley
#     output_folder=output_folder_ga2,
# )
# GrowthAssayPlotlib.plot_pretty_barplot(
#     df=df2_melted,
#     plot_clones=True, # or 'Line' if you want clones to show separatley
#     output_folder=output_folder_ga2,
# )

# clone_list = [
#     'KOEBA181 C1',
#     'KORH4 C1',
#     'KOEBA140 C4', 
#     'KOEBA175 C6',
#     'KORH2a C3',
#     'KORh1 C2',
#     'KOPfs25 C1',
#     'KOP230P C3',
# ]
# dfq = df2_melted.query('Line == @clone_list')
# GrowthAssayPlotlib.plot_pretty_barplot(
#     df=dfq,
#     plot_clones=True, # or 'Line' if you want clones to show separatley
#     output_folder=output_folder_ga2,
# )


In [ ]:
GrowthAssayPlotlib.significance_testing(
    df=df2_melted_grouped, 
    group_keys=['Strain', 'Speed', 'Hematocrit'],
    line_key='Strain',
    wildtype='NF54',
)

In [ ]:
import re

colors_light = {
    'NF54': '#829AA3',
    'Heparin 50':'#829AA3',
    
    '3D7':'#B9BDC5',
    '3D7 (viola)': '#B9BDC5',

    'KOP230P': '#91b1be',
    'KOPfs25': '#9fc9d9',

    'KOEBA140': '#DAEBC6',
    'KOEBA181': '#84B06D',
    'KOEBA175': '#C1E2D4',
    'KOEBA175+Anti-GYPA':'#A5D0D4',
    'KORH1': '#F9DA9A',
    'KORH4': '#F3A59B',
    'KORh2a': '#F7CAA1',
    'KORH2a': '#F7CAA1',

    

    'Barseq': '#BEBFBC',

    'cKOMSP1 DMSO':'#B9BDC5',
    'cKOMSP1 Rap':'#FFE3B8',

    'Neuraminidase':'#D6EE96',
    
    'Anti-CD55':'#97B6DA',
    'Anti-GYPA':'#B6E2DF',
    'R1':'#F1AAD1',
    'Anti-GYPC':'#93E1DB',
    'Anti-CR1 ab25':'#DEAC7F',
    'Anti-CR1 ABIN':'#DEAC7F',

    'Anti-Basigin':'#C6C5D3',
   
    
  
    'cKOGAP45 DMSO': '#B9BDC5',
    'cKOGAP45 Rap': '#E5486C',

    'cKOAMA1 DMSO':'#B9BDC5',
    'cKOAMA1 Rap':'#EBD1D9',

    'NF54 x': '#829AA3',
    'NF54 y': '#829AA3',
    'NF54 mean': '#829AA3',
    'NF54 single': '#829AA3',

    'cKOMSP1 x': '#B9BDC5',
    'cKOMSP1 y': '#B9BDC5',
    'cKOMSP1 mean': '#B9BDC5',
    'cKOMSP1 single': '#B9BDC5',

    'cKOAMA1 x': '#B9BDC5',
    'cKOAMA1 y': '#B9BDC5',
    'cKOAMA1 mean': '#B9BDC5',
    'cKOAMA1 single': '#B9BDC5',

    'cKOGAP45 x': '#B9BDC5',
    'cKOGAP45 y': '#B9BDC5',
    'cKOGAP45 mean': '#B9BDC5',
    'cKOGAP45 single': '#B9BDC5',
    
 
}

colors_dark = {
    'NF54': '#043546',
    'Heparin 50':'#818897',

    '3D7':'#818897',
    '3D7 (viola)': '#818897',

    'KOP230P': '#406a7d',
    'KOPfs25': '#78a3b9',

    'KOEBA140': '#9ECA69',
    'KOEBA181': '#486737',
    'KOEBA175': '#6AB997',
    'KOEBA175+Anti-GYPA':'#04A0AF',
    'KORH1': '#F1AF23',
    'KORH4': '#E8513E',
    'KORh2a': '#ED8626',
    'KORH2a': '#ED8626',


    'Barseq': '#BEBFBC',

    'cKOMSP1 DMSO':'#FFD086',
    'cKOMSP1 Rap':'#FFD086',

    'Neuraminidase':'#ABDC29',
    
    'Anti-CD55':'#97B6DA',
    'Anti-GYPA':'#91D4CF',
    'R1':'#E459A5',
    'Anti-GYPC':'#35BEB4',
    'Anti-CR1 ab25':'#C6545E',
    'Anti-CR1 ABIN':'#C6545E',

    'Anti-Basigin':'#7E7C9C',
   
    
  
    'cKOGAP45 DMSO': '#81132C',
    'cKOGAP45 Rap': '#81132C',

    
    'cKOAMA1 DMSO':'#CC889D',
    'cKOAMA1 Rap':'#CC889D',

    'NF54 x': '#043546',
    'NF54 y': '#043546',
    'NF54 mean': '#043546',
    'NF54 single': '#043546',

    'cKOMSP1 x': '#FFD086',
    'cKOMSP1 y': '#FFD086',
    'cKOMSP1 mean': '#FFD086',
    'cKOMSP1 single': '#FFD086',

    'cKOAMA1 x': '#CC889D',
    'cKOAMA1 y': '#CC889D',
    'cKOAMA1 mean': '#CC889D',
    'cKOAMA1 single': '#CC889D',

    'cKOGAP45 x': '#81132C',
    'cKOGAP45 y': '#81132C',
    'cKOGAP45 mean': '#81132C',
    'cKOGAP45 single': '#81132C',
}

# OT paper Fig 1
pattern = f'({"|".join(hematocrit_levels)})$'
df_hem['Line_clone'] = df_hem['Line'].apply(lambda x: re.sub(pattern, '', x).strip())

print(df_hem['Line_clone'].unique())

speed_order_filter = ['Static']
line_order_filter = ['NF54','3D7']

dfq = df_hem
dfq = dfq.query('Hematocrit == "4%"')
dfq = dfq.query('Speed == @speed_order_filter')
dfq = dfq.query('Line_clone == @line_order_filter')

plt.figure(figsize=(4,9), dpi=300)

sns.set_theme(#context='notebook', 
            #style='ticks', 
            style='darkgrid', 
            font='arial', 
            font_scale=1) 
            #   color_codes=True, 
            #   rc=None)
sns.set_style(rc = {'axes.facecolor': '#F5F5F9'})

sns.boxplot(x='Line_clone', 
            order=line_order_filter, 
            y='Invasion rate', data=dfq, 
            hue='Strain',  
            palette=colors_light, 
            dodge=False,
            medianprops={"color": "red"},
            width=0.6,
            )


sns.stripplot(x='Line_clone', 
              order=line_order_filter, 
              y='Invasion rate', 
              data=dfq,
              hue='Strain',
              palette=colors_dark, 
              dodge=False, 
              size=7, 
              legend=False,
              jitter=True, 
              )

#plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
#plt.title(f'Static vs 90-rpm')
plt.ylim((0,14))
plt.xticks(rotation=90)
plt.xlabel('')
plt.ylabel('Growth rate (PEMR)')
plt.savefig(os.path.join(output_folder_ga2, f'NF54vs3D7_static.{GrowthAssayPlotlib.EXTENSION}'), bbox_inches='tight')
plt.legend().remove()
# plt.close()

# with pd.option_context('display.max_rows', None,
#                        'display.max_columns', None,
#                        'display.precision', 3,
#                        ):
#     print(df_hem)

def sem(data):
    return np.std(data, ddof=1) / np.sqrt(np.size(data))

for (speed, line), dfg in df_hem.groupby(['Speed','Line',]):
    data = dfg['Invasion rate']
    print(f'{speed}, {line}, {np.mean(data):.2f}, {sem(data):.2f},{np.size(data)}')

# this allows the text to be editable in affinitity (it rescales the text)
plt.rcParams['svg.fonttype'] = 'none'
plt.savefig(os.path.join(output_folder_ga2, f'tweezer attachment force_S1.svg'), bbox_inches='tight')

print({len(dfq.query('Strain == "3D7"'))})
